In [1]:
import os
# You can use `tensorflow`, `pytorch`, `jax` here
# KerasCore makes the notebook backend agnostic :)
os.environ["KERAS_BACKEND"] = "tensorflow"


import keras
from keras import layers

import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

# %% [markdown]
# # Configuration
# 
# A particularly good practise is to have a configuration class for your notebooks. This not only keeps your configurations all at a single place but also becomes handy to map the configs to the performance of the model.
# 
# Please play around with the configurations and see how the performance of the model changes.

# %% [markdown]
# ## Note on some observations
# 
# Reference Notebook: https://www.kaggle.com/code/aritrag/eda-train-csv
# 
# 1. Class Dependencies: Refers to inherent relationships between classes in the analysis.
# 2. Complementarity: `bowel_injury` and `bowel_healthy`, as well as `extravasation_injury` and `extravasation_healthy`, are perfectly complementary, with their sum always equal to 1.0.
# 3. Simplification: For the model, only `{bowel/extravasation}_injury` will be included, and the corresponding healthy status can be calculated using a sigmoid function.
# 4. Softmax: `{kidney/liver/spleen}_{healthy/low/high}` classifications are softmaxed, ensuring their combined probabilities sum up to 1.0 for each organ, simplifying the model while preserving essential information.

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:23.231676Z","iopub.execute_input":"2023-09-09T19:02:23.232042Z","iopub.status.idle":"2023-09-09T19:02:23.247729Z","shell.execute_reply.started":"2023-09-09T19:02:23.232007Z","shell.execute_reply":"2023-09-09T19:02:23.246654Z"}}
class Config:
    SEED = 42
    IMAGE_SIZE = [224, 224]
    BATCH_SIZE = 32
    EPOCHS = 50
    TARGET_COLS  = [
        "bowel_injury", "extravasation_injury",
        "kidney_healthy", "kidney_low", "kidney_high",
        "liver_healthy", "liver_low", "liver_high",
        "spleen_healthy", "spleen_low", "spleen_high",
    ]
    AUTOTUNE = tf.data.AUTOTUNE

config = Config()

# %% [markdown]
# # Reproducibility
# 
# We would want this notebook to have reproducible results. Here we set the seed for all the random algorithms so that we can reproduce the experiments each time exactly the same way.

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:23.251220Z","iopub.execute_input":"2023-09-09T19:02:23.251610Z","iopub.status.idle":"2023-09-09T19:02:23.298498Z","shell.execute_reply.started":"2023-09-09T19:02:23.251581Z","shell.execute_reply":"2023-09-09T19:02:23.297532Z"}}
keras.utils.set_random_seed(seed=config.SEED)

# %% [markdown]
# # Dataset
# 
# The dataset provided in the competition consists of DICOM images. We will not be training on the DICOM images, rather would work on PNG image which are extracted from the DICOM format.
# 
# [A helpful resource on the conversion of DICOM to PNG](https://www.kaggle.com/code/radek1/how-to-process-dicom-images-to-pngs)

# %% [code] {"_kg_hide-input":true,"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:23.304536Z","iopub.execute_input":"2023-09-09T19:02:23.304911Z","iopub.status.idle":"2023-09-09T19:02:23.310205Z","shell.execute_reply.started":"2023-09-09T19:02:23.304884Z","shell.execute_reply":"2023-09-09T19:02:23.308909Z"}}
BASE_PATH = f"D:/RSNA_Project/archive"

# %% [markdown]
# ## Meta Data
# 
# The `train.csv` file contains the following meta information:
# 
# - `patient_id`: A unique ID code for each patient.
# - `series_id`: A unique ID code for each scan.
# - `instance_number`: The image number within the scan. The lowest instance number for many series is above zero as the original scans were cropped to the abdomen.
# - `[bowel/extravasation]_[healthy/injury]`: The two injury types with binary targets.
# - `[kidney/liver/spleen]_[healthy/low/high]`: The three injury types with three target levels.
# - `any_injury`: Whether the patient had any injury at all.

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:23.311914Z","iopub.execute_input":"2023-09-09T19:02:23.313537Z","iopub.status.idle":"2023-09-09T19:02:23.443167Z","shell.execute_reply.started":"2023-09-09T19:02:23.313500Z","shell.execute_reply":"2023-09-09T19:02:23.441731Z"}}
# train
dataframe = pd.read_csv(f"{BASE_PATH}/train.csv")
dataframe["image_path"] = f"{BASE_PATH}/train_images"\
                    + "/" + dataframe.patient_id.astype(str)\
                    + "/" + dataframe.series_id.astype(str)\
                    + "/" + dataframe.instance_number.astype(str) +".png"
dataframe = dataframe.drop_duplicates()

dataframe.head(2)

# %% [markdown]
# We split the training dataset into train and validation. This is a common practise in the Machine Learning pipelines. We not only want to train our model, but also want to validate it's training.
# 
# A small catch here is that the training and validation data should have an aligned data distribution. Here we handle that by grouping the lables and then splitting the dataset. This ensures an aligned data distribution between the training and the validation splits.

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:23.446378Z","iopub.execute_input":"2023-09-09T19:02:23.446699Z","iopub.status.idle":"2023-09-09T19:02:23.562238Z","shell.execute_reply.started":"2023-09-09T19:02:23.446671Z","shell.execute_reply":"2023-09-09T19:02:23.560740Z"}}
# Function to handle the split for each group
def split_group(group, test_size=0.25):
    if len(group) == 1:
        return (group, pd.DataFrame()) if np.random.rand() < test_size else (pd.DataFrame(), group)
    else:
        return train_test_split(group, test_size=test_size, random_state=42)

# Initialize the train and validation datasets
train_data = pd.DataFrame()
val_data = pd.DataFrame()

# Iterate through the groups and split them, handling single-sample groups
for _, group in dataframe.groupby(config.TARGET_COLS):
    train_group, val_group = split_group(group)
    train_data = pd.concat([train_data, train_group], ignore_index=True)
    val_data = pd.concat([val_data, val_group], ignore_index=True)

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:23.564011Z","iopub.execute_input":"2023-09-09T19:02:23.564429Z","iopub.status.idle":"2023-09-09T19:02:23.572678Z","shell.execute_reply.started":"2023-09-09T19:02:23.564388Z","shell.execute_reply":"2023-09-09T19:02:23.571464Z"}}
train_data.shape, val_data.shape

# %% [markdown]
# ## Data Pipeline /w tf.data
# 
# Here we build the data pipeline using `tf.data`. Using `tf.data` we can map out data to an augmentation pipeline simple by using the ` map` API.
# 
# Adding augmentations to the data pipeline is as simple as adding a layer into the list of layers that the `Augmenter` processes.
# 
# Reference: https://keras.io/api/keras_cv/layers/augmentation/

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:23.574325Z","iopub.execute_input":"2023-09-09T19:02:23.575215Z","iopub.status.idle":"2023-09-09T19:02:23.586203Z","shell.execute_reply.started":"2023-09-09T19:02:23.575161Z","shell.execute_reply":"2023-09-09T19:02:23.585314Z"}}
def decode_image_and_label(image_path, label):
    file_bytes = tf.io.read_file(image_path)
    image = tf.io.decode_png(file_bytes, channels=3, dtype=tf.uint8)
    image = tf.image.resize(image, config.IMAGE_SIZE, method="bilinear")
    image = tf.cast(image, tf.float32) / 255.0
    
    label = tf.cast(label, tf.float32)
    #         bowel       fluid       kidney      liver       spleen
    labels = (label[0:1], label[1:2], label[2:5], label[5:8], label[8:11])
    
    return (image, labels)


# def apply_augmentation(images, labels):
#     augmenter = keras_cv.layers.Augmenter(
#         [
#             keras_cv.layers.RandomFlip(mode="horizontal_and_vertical"),
#             keras_cv.layers.RandomCutout(height_factor=0.2, width_factor=0.2),
            
#         ]
#     )
#     return (augmenter(images), labels)


def build_dataset(image_paths, labels):
    num_validation_samples = len(image_paths)
    
    # Calculate the number of batches required for the given batch size
    num_batches = num_validation_samples // config.BATCH_SIZE

    # Calculate the new total number of samples after truncation
    new_num_samples = (num_batches) * config.BATCH_SIZE
    
    # Truncate the validation data to the new size
    image_paths = image_paths[:new_num_samples]
    labels = labels[:new_num_samples]
    ds = (
        tf.data.Dataset.from_tensor_slices((image_paths, labels))
        .map(decode_image_and_label, num_parallel_calls=config.AUTOTUNE)
        .shuffle(config.BATCH_SIZE * 10)
        .batch(config.BATCH_SIZE)
#         .map(apply_augmentation, num_parallel_calls=config.AUTOTUNE)
        .prefetch(config.AUTOTUNE)
    )
    return ds

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:23.587583Z","iopub.execute_input":"2023-09-09T19:02:23.588024Z","iopub.status.idle":"2023-09-09T19:02:24.523971Z","shell.execute_reply.started":"2023-09-09T19:02:23.587992Z","shell.execute_reply":"2023-09-09T19:02:24.522907Z"}}
paths  = train_data.image_path.tolist()
labels = train_data[config.TARGET_COLS].values

ds = build_dataset(image_paths=paths, labels=labels)
images, labels = next(iter(ds))
images.shape, [label.shape for label in labels]

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:24.525330Z","iopub.execute_input":"2023-09-09T19:02:24.525691Z","iopub.status.idle":"2023-09-09T19:02:24.895828Z","shell.execute_reply.started":"2023-09-09T19:02:24.525657Z","shell.execute_reply":"2023-09-09T19:02:24.894736Z"}}
# No more customizing your plots by hand, KerasCV has your back ;)

# %% [markdown]
# # Build Model
# 
# We are going to load a pretrained model from the [list of avaiable backbones in KerasCV](https://keras.io/api/keras_cv/models/backbones/). We are using the `ResNetBackbone` as our backbone. The practise of using a pretrained model and finetuning it to a specific dataset is prevalent in the DL community.
# 
# We use the [Functional API](https://keras.io/guides/functional_api/) of Keras to build the model. The design of the model would be such that we input a single image and we get different heads for the various predictions we need (kidney, spleen...).
# 
# We have also added a Learning Rate scheduler for you to work with. When an athlete trains, the first step is always to warm up. We take a similar approach to training our models. We warm up with model where the learning rate increses from the initial LR to a higher LR. After the warmup stage we provide a decay algorithm (cosine here). A list of all the learning rate scheduler can be found [here](https://keras.io/api/optimizers/learning_rate_schedules/).

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:24.897740Z","iopub.execute_input":"2023-09-09T19:02:24.898547Z","iopub.status.idle":"2023-09-09T19:02:24.927962Z","shell.execute_reply.started":"2023-09-09T19:02:24.898507Z","shell.execute_reply":"2023-09-09T19:02:24.927231Z"}}
def build_model(warmup_steps, decay_steps):
    # Define Input
    inputs = keras.Input(shape=config.IMAGE_SIZE + [3,], batch_size=config.BATCH_SIZE,dtype='float32')
    print(inputs.shape)
    # Define Backbone
#     backbone = keras_cv.models.ResNetBackbone.from_preset("resnet50_imagenet")
#     backbone.include_rescaling = False
#     x = backbone(inputs)
    x1 = keras.layers.Conv2D(8,(3,3),activation = 'relu',padding = 'same',dtype='float32')(inputs)
    x1 = keras.layers.MaxPool2D((2,2),dtype='float32')(x1)
    x1 = keras.layers.Conv2D(16,(3,3),activation = 'relu',padding = 'same',dtype='float32')(inputs)
    x1 = keras.layers.MaxPool2D((2,2),dtype='float32')(x1)
    x1 = keras.layers.Conv2D(32,(3,3),activation = 'relu',padding = 'same',dtype='float32')(x1)
    x1 = keras.layers.MaxPool2D((2,2),dtype='float32')(x1)
    x1 = keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'same',dtype='float32')(x1)
    x1 = keras.layers.MaxPool2D((2,2),dtype='float32')(x1)
    x = keras.layers.Flatten(dtype='float32')(x1)
    x = keras.layers.Dropout(0.5)(x)
    # GAP to get the activation maps
#     gap = keras.layers.GlobalAveragePooling2D()
#     x = gap(x)

    
    #extraversion model
    
    x2 = keras.layers.Dense(128,activation= 'silu',dtype='float32')(x)
    x2 = keras.layers.Dropout(0.5,dtype='float32')(x2)
    x2 = keras.layers.Dense(64,activation= 'silu',dtype='float32')(x2)
    x2 = keras.layers.Dropout(0.5,dtype='float32')(x2)
    x2 = keras.layers.Dense(32,activation= 'silu',dtype='float32')(x2)
    x2 = keras.layers.Dropout(0.5,dtype='float32')(x2)
    x_extra = keras.layers.Dense(16, activation='silu',dtype='float32')(x2)
    x_extra = keras.layers.Dropout(0.5,dtype='float32')(x_extra)
    out_extra = keras.layers.Dense(1, name='extra', activation='sigmoid',dtype='float32')(x_extra) # use sigmoid to convert predictions to [0-1]
    
    #liver model
    x_liver = keras.layers.Dense(16, activation='silu',dtype='float32')(x)
    x_liver = keras.layers.Dropout(0.5,dtype='float32')(x_liver)
    out_liver = keras.layers.Dense(3, name='liver', activation='softmax',dtype='float32')(x_liver) # use softmax for the liver head
    
    #kidney model
    x_kidney = keras.layers.Dense(32, activation='silu',dtype='float32')(x)
    x_kidney = keras.layers.Dropout(0.5,dtype='float32')(x_kidney)
    x_kidney = keras.layers.Dense(16, activation='silu',dtype='float32')(x_kidney )
    x_kidney = keras.layers.Dropout(0.5,dtype='float32')(x_kidney)
    out_kidney = keras.layers.Dense(3, name='kidney', activation='softmax',dtype='float32')(x_kidney) # use softmax for the kidney head
    
    #spleen model
    x_spleen = keras.layers.Dense(32,activation="relu",dtype='float32')(x)
    x_spleen = keras.layers.Dropout(0.5,dtype='float32')(x_spleen)
    x_spleen = keras.layers.Dense(16, activation='silu',dtype='float32')(x_spleen)
    x_spleen = keras.layers.Dropout(0.5,dtype='float32')(x_spleen)
    out_spleen = keras.layers.Dense(3, name='spleen', activation='softmax',dtype='float32')(x_spleen) # use softmax for the spleen head
    
    #bowel model
#     x_bowel = keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'same',dtype='float64')(x1)
#     x_bowel = keras.layers.MaxPool2D((2,2),dtype='float64')(x_bowel)
#     x_bowel = keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'same',dtype='float64')(x_bowel)
#     x_bowel = keras.layers.MaxPool2D((2,2),dtype='float64')(x_bowel)
#     x_bowel = keras.layers.Flatten(dtype='float64')(x_bowel)
#     x_bowel = keras.layers.Dropout(0.5)(x_bowel)
#     x_bowel = keras.layers.Dense(128,activation= 'relu',dtype='float64')(x_bowel)
#     x_bowel = keras.layers.Dropout(0.5)(x_bowel)
#     x_bowel = keras.layers.Dense(16, activation='relu')(x_bowel)
#     x_bowel = keras.layers.Dropout(0.5)(x_bowel)
#     backbone = keras_cv.models.ResNetBackbone.from_preset("resnet50_imagenet")
#     backbone.include_rescaling = False
#     x_bowel = backbone(inputs)
    x_bowel = keras.layers.Flatten(dtype='float32')(x1)
    x_bowel = keras.layers.Dropout(0.5,dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dense(128,activation= 'silu',dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dropout(0.5,dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dense(64,activation= 'silu',dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dropout(0.5,dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dense(64,activation= 'silu',dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dropout(0.5,dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dense(32,activation= 'silu',dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dropout(0.5,dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dense(32,activation= 'silu',dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dropout(0.5,dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dense(32,activation= 'silu',dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dropout(0.5,dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dense(32,activation= 'silu',dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dropout(0.5,dtype='float32')(x_bowel)
    x_bowel = keras.layers.Dense(16,activation= 'silu',dtype='float32')(x_bowel)
#     x_bowel = keras.layers.Dropout(0.5,dtype='float32')(x_bowel)
    
    out_bowel = keras.layers.Dense(1, name='bowel', activation='sigmoid',dtype='float32')(x_bowel) # use sigmoid to convert predictions to [0-1]
    
    
    # Concatenate the outputs
    outputs = [out_bowel, out_extra, out_liver, out_kidney, out_spleen]

    # Create model
    print("[INFO] Building the model...")
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    # Cosine Decay
#     cosine_decay = keras.optimizers.schedules.CosineDecay(
#         initial_learning_rate=1e-4,
#         decay_steps=decay_steps,
#         alpha=0.0,
#         warmup_target=1e-3,
#         warmup_steps=warmup_steps,
#     )
    initial_learning_rate = 1e-5
    cosine_decay = tf.keras.experimental.CosineDecay(
    initial_learning_rate, decay_steps)

    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=cosine_decay)
    loss = {
        "bowel":keras.losses.BinaryCrossentropy(),
        "extra":keras.losses.BinaryCrossentropy(),
        "liver":keras.losses.CategoricalCrossentropy(),
        "kidney":keras.losses.CategoricalCrossentropy(),
        "spleen":keras.losses.CategoricalCrossentropy(),
    }
    metrics = {
        "bowel":["accuracy"],
        "extra":["accuracy"],
        "liver":["accuracy"],
        "kidney":["accuracy"],
        "spleen":["accuracy"],
    }
    print("[INFO] Compiling the model...")
    model.compile(
        optimizer=optimizer,
      loss=loss,
      metrics=metrics
    )
    
    return model

# %% [markdown]
# # Train the model with "model.fit"

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:24.929689Z","iopub.execute_input":"2023-09-09T19:02:24.930568Z","iopub.status.idle":"2023-09-09T19:02:25.036305Z","shell.execute_reply.started":"2023-09-09T19:02:24.930528Z","shell.execute_reply":"2023-09-09T19:02:25.035235Z"}}
# get image_paths and labels
print("[INFO] Building the dataset...")
train_paths = train_data.image_path.values; train_labels = train_data[config.TARGET_COLS].values.astype(np.float32)
valid_paths = val_data.image_path.values; valid_labels = val_data[config.TARGET_COLS].values.astype(np.float32)

# train and valid dataset
train_ds = build_dataset(image_paths=train_paths, labels=train_labels)
val_ds = build_dataset(image_paths=valid_paths, labels=valid_labels)

total_train_steps = train_ds.cardinality().numpy() * config.BATCH_SIZE * config.EPOCHS
warmup_steps = int(total_train_steps * 0.10)
decay_steps = total_train_steps - warmup_steps

print(f"{total_train_steps=}")
print(f"{warmup_steps=}")
print(f"{decay_steps=}")

# %% [code] {"_kg_hide-input":true,"_kg_hide-output":false,"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2023-09-09T19:02:25.040508Z","iopub.execute_input":"2023-09-09T19:02:25.040847Z"}}
# build the model
print("[INFO] Building the model...")
model = build_model(warmup_steps, decay_steps)

# train
print("[INFO] Training...")
history = model.fit(
    train_ds,
    epochs=config.EPOCHS,
    validation_data=val_ds,
)

# %% [markdown]
# ## Visualize the training plots

# %% [code] {"jupyter":{"outputs_hidden":false}}
# Create a 3x2 grid for the subplots
fig, axes = plt.subplots(5, 1, figsize=(5, 15))

# Flatten axes to iterate through them
axes = axes.flatten()

# Iterate through the metrics and plot them
for i, name in enumerate(["bowel", "extra", "kidney", "liver", "spleen"]):
    # Plot training accuracy
    axes[i].plot(history.history[name + '_accuracy'], label='Training ' + name)
    # Plot validation accuracy
    axes[i].plot(history.history['val_' + name + '_accuracy'], label='Validation ' + name)
    axes[i].set_title(name)
    axes[i].set_xlabel('Epoch')
    axes[i].set_ylabel('Accuracy')
    axes[i].legend()

plt.tight_layout()
plt.show()

# %% [code] {"jupyter":{"outputs_hidden":false}}
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val loss")
plt.legend()
plt.show()

# %% [code] {"jupyter":{"outputs_hidden":false}}
# store best results
best_epoch = np.argmin(history.history['val_loss'])
best_loss = history.history['val_loss'][best_epoch]
best_acc_bowel = history.history['val_bowel_accuracy'][best_epoch]
best_acc_extra = history.history['val_extra_accuracy'][best_epoch]
best_acc_liver = history.history['val_liver_accuracy'][best_epoch]
best_acc_kidney = history.history['val_kidney_accuracy'][best_epoch]
best_acc_spleen = history.history['val_spleen_accuracy'][best_epoch]

# Find mean accuracy
best_acc = np.mean(
    [best_acc_bowel,
     best_acc_extra,
     best_acc_liver,
     best_acc_kidney,
     best_acc_spleen
])


print(f'>>>> BEST Loss  : {best_loss:.3f}\n>>>> BEST Acc   : {best_acc:.3f}\n>>>> BEST Epoch : {best_epoch}\n')
print('ORGAN Acc:')
print(f'  >>>> {"Bowel".ljust(15)} : {best_acc_bowel:.3f}')
print(f'  >>>> {"Extravasation".ljust(15)} : {best_acc_extra:.3f}')
print(f'  >>>> {"Liver".ljust(15)} : {best_acc_liver:.3f}')
print(f'  >>>> {"Kidney".ljust(15)} : {best_acc_kidney:.3f}')
print(f'  >>>> {"Spleen".ljust(15)} : {best_acc_spleen:.3f}')

# %% [markdown]
# ## Store the model for inference

# %% [code] {"jupyter":{"outputs_hidden":false}}
# Save the model
model.save("rsna-arch1_4.h5")


[INFO] Building the dataset...
total_train_steps=449600
warmup_steps=44960
decay_steps=404640
[INFO] Building the model...
(32, 256, 256, 3)
[INFO] Building the model...
[INFO] Compiling the model...
[INFO] Training...
Epoch 1/50
281/281 [==============================] - 31s 91ms/step - loss: 3.6688 - bowel_loss: 0.6995 - extra_loss: 0.6667 - liver_loss: 0.5796 - kidney_loss: 0.7500 - spleen_loss: 0.9729 - bowel_accuracy: 0.4884 - extra_accuracy: 0.6393 - liver_accuracy: 0.8421 - kidney_accuracy: 0.7494 - spleen_accuracy: 0.5737 - val_loss: 3.4866 - val_bowel_loss: 0.6931 - val_extra_loss: 0.6592 - val_liver_loss: 0.6409 - val_kidney_loss: 0.5878 - val_spleen_loss: 0.9055 - val_bowel_accuracy: 0.4814 - val_extra_accuracy: 0.6955 - val_liver_accuracy: 0.8684 - val_kidney_accuracy: 0.8105 - val_spleen_accuracy: 0.6765
Epoch 2/50
281/281 [==============================] - 25s 90ms/step - loss: 3.6602 - bowel_loss: 0.6930 - extra_loss: 0.6566 - liver_loss: 0.6312 - kidney_loss: 0.7080 - s

KeyboardInterrupt: 